# Functions used for simulating different models.

# Heston Stock+Swap

In [1]:
#We probably want to do Full Truncation Scheme.
#implement this in a function and run NN's again.
# #Full Truncation
# #for i in range(steps): 
# #    X = np.log(S[:,i])
# #    V = kb.maximum(vol[:,i],0)# V^+
# #    X = X+(r-0.5*V)*dt+np.sqrt(V)*dW1[:,i]
# #    S[:,i+1] = np.exp(X)#stock process
# #    vol[:,i+1] = vol[:,i]+kappa*(theta-V)*dt+sigma*np.sqrt(V)*dW2[:,i]#vol process

def HestonFullTruncation_SV(n):
    #change in BM
    dW1=npr.normal(0,np.sqrt(dt),(n,steps))#Change in first BM
    dW2=rho*dW1+np.sqrt(1-rho**2)*npr.normal(0,np.sqrt(dt),(n,steps))#change in second BM such that 

    #vol and S
    S = np.empty([n,steps+1])
    vol = np.empty([n,steps+1])
    S[:,0] = S0
    vol[:,0] = V0

    #S2 prices and intermediate calculations
    L_matrix = np.empty([n,steps+1])
    Int_V = np.empty([n,steps+1])
    S2 = np.empty([n,steps+1])

    L_matrix[:,0] = L(0,V0)
    Int_V[:,0] = 0
    S2[:,0] = L(0,vol[:,0])#initial price of Swap. As we are at time 0 int=0, s.t. we only have L(0,V_0)

    #¤Consider simulating V at more time steps in order to get a better approximation of int V for the price of S2
    for i in range(steps):
               
        X = np.log(S[:,i])
        V = np.maximum(vol[:,i],0)# V^+
        X = X+(r-0.5*V)*dt+np.sqrt(V)*dW1[:,i]
        S[:,i+1] = np.exp(X)#stock process
        vol[:,i+1] = vol[:,i]+kappa*(theta-V)*dt+sigma*np.sqrt(V)*dW2[:,i]#vol process
        
        L_matrix[:,i+1] = L(dt*(i+1),vol[:,i])
        Int_V[:,i+1] = Int_V[:,i]+dt*(vol[:,i]+vol[:,i+1])/2#by the trapzoid rule
        S2[:,i+1] = L_matrix[:,i+1]+Int_V[:,i+1]
    
    return S,vol,S2    



#According to Deep Hedging then vol process can be simulated exactly. 
#Maybe explore this.
def HestonMomentMatching_SV(n):
    #change in BM
    dW1=npr.normal(0,np.sqrt(dt),(n,steps))#Change in first BM
    dW2=rho*dW1+np.sqrt(1-rho**2)*npr.normal(0,np.sqrt(dt),(n,steps))#change in second BM such that 

    #vol and S
    S = np.empty([n,steps+1])
    vol = np.empty([n,steps+1])
    S[:,0] = S0
    vol[:,0] = V0

    #S2 prices and intermediate calculations
    L_matrix = np.empty([n,steps+1])
    Int_V = np.empty([n,steps+1])
    S2 = np.empty([n,steps+1])

    L_matrix[:,0] = L(0,V0)
    Int_V[:,0] = 0
    S2[:,0] = L(0,vol[:,0])#initial price of Swap. As we are at time 0 int=0, s.t. we only have L(0,V_0)

    #¤Consider simulating V at more time steps in order to get a better approximation of int V for the price of S2
    for i in range(steps):
        X = np.log(S[:,i])
        V = vol[:,i]# just to have it in short notation.
        X = X+(r-0.5*V)*dt+np.sqrt(V)*dW1[:,i]
        S[:,i+1] = np.exp(X) #stock process 
        Gam = np.sqrt(1/dt*np.log(1+(1/2*sigma**2/kappa*V*(1-np.exp(-2*kappa*dt)))/((np.exp(-kappa*dt)*V+(1-np.exp(-kappa*dt))*theta)**2)))
        vol[:,i+1] = (np.exp(-kappa*dt)*V+(1-np.exp(-kappa*dt))*theta)*np.exp(-1/2*Gam**2*dt+Gam*dW2[:,i])
        L_matrix[:,i+1] = L(dt*(i+1),vol[:,i])
        Int_V[:,i+1] = Int_V[:,i]+dt*(vol[:,i]+vol[:,i+1])/2#by the trapzoid rule
        S2[:,i+1] = L_matrix[:,i+1]+Int_V[:,i+1]
    
    return S,vol,S2

#¤consider if we might just want to have one function or let the second function call the first. 
def DataInputOutput_HSV(S,vol,S2):
    #dataset used for output
    dS1 = np.diff(S,1,1)#calculates stock price changes
    dS2 = np.diff(S2,1,1)#calculates S2 price changes (Swap)
    dS = np.concatenate((dS1,dS2),1)
    
    #Rearranges dataset such that we get dS1,dS2,dS1,dS2,... in each observation insted of dS1,dS1,...,dS1,dS2,dS2,...,dS2
    idx=np.insert(range(steps,steps*2),np.arange(steps), range(steps))
    for i in range(n):
        dS[i]=dS[i][idx]
    
    
    #dataset used for input
    tSinput = []
    tim = np.linspace(0,T,steps+1)#Vector of timepoint where pf i adjusted.
    
    for i in range(steps):
        timv = np.repeat(tim[i],n)
        timv = np.reshape(timv,(n,1))
        logSv = np.reshape(np.log(S[:,i]),(n,1))
        volv = np.reshape(vol[:,i],(n,1))
        #just a test of not inputting t.
        #tSinput.append(np.concatenate((logSv,volv),1))#Input is (S,V)
        tSinput.append(np.concatenate((timv,logSv,volv),1))#Input is (t,S,V)
    #tSinput=np.array(tSinput)
    return tSinput,dS

#data structured for RNN
def RNN_DataInputOutput_HSV(S,vol,S2):
    #output
    dS1 = np.diff(S,1,1)#calculates stock price changes
    dS2 = np.diff(S2,1,1)#calculates S2 price changes (Swap)
    dS = np.concatenate((dS1,dS2),1)
    
    #Rearranges dataset such that we get dS1,dS2,dS1,dS2,... in each observation insted of dS1,dS1,...,dS1,dS2,dS2,...,dS2
    idx=np.insert(range(steps,steps*2),np.arange(steps), range(steps))
    for i in range(n):
        dS[i]=dS[i][idx]
    dS=np.reshape(dS,(100000,30,2))
    
    #input
    tSinput = []
    tim = np.linspace(0,T,steps+1)

    logS=np.log(S)
    for i in range(n):
        tmp_input=[]
        for j in range(steps):
            tmp_input.append([tim[j],logS[i][j],vol[i][j]])
        tSinput.append(tmp_input)
    tSinput=np.array(tSinput)
    
    return tSinput,dS

# Heston only stock

In [2]:
#According to Deep Hedging then vol process can be simulated exactly. 
#Maybe explore this.
def HestonMomentMatching_S(n):
    #change in BM
    dW1=npr.normal(0,np.sqrt(dt),(n,steps))#Change in first BM
    dW2=rho*dW1+np.sqrt(1-rho**2)*npr.normal(0,np.sqrt(dt),(n,steps))#change in second BM such that 

    #vol and S
    S = np.empty([n,steps+1])
    vol = np.empty([n,steps+1])
    S[:,0] = S0
    vol[:,0] = V0

    #S2 prices and intermediate calculations
    L_matrix = np.empty([n,steps+1])
    Int_V = np.empty([n,steps+1])
    S2 = np.empty([n,steps+1])

    L_matrix[:,0] = L(0,V0)
    Int_V[:,0] = 0
    S2[:,0] = L(0,vol[:,0])#initial price of Swap. As we are at time 0 int=0, s.t. we only have L(0,V_0)

    #¤Consider simulating V at more time steps in order to get a better approximation of int V for the price of S2
    for i in range(steps):
        X = np.log(S[:,i])
        V = vol[:,i]# just to have it in short notation.
        X = X+(r-0.5*V)*dt+np.sqrt(V)*dW1[:,i]
        S[:,i+1] = np.exp(X)#stock process
        Gam = np.sqrt(1/dt*np.log(1+(1/2*sigma**2/kappa*V*(1-np.exp(-2*kappa*dt)))/((np.exp(-kappa*dt)*V+(1-np.exp(-kappa*dt))*theta)**2)))
        vol[:,i+1] = (np.exp(-kappa*dt)*V+(1-np.exp(-kappa*dt))*theta)*np.exp(-1/2*Gam**2*dt+Gam*dW2[:,i])
        L_matrix[:,i+1] = L(dt*(i+1),vol[:,i])
        Int_V[:,i+1] = Int_V[:,i]+dt*(vol[:,i]+vol[:,i+1])/2#by the trapzoid rule
        S2[:,i+1] = L_matrix[:,i+1]+Int_V[:,i+1]
    
    return S

def DataInputOutput_HS(S):
    #dataset used for output
    dS = np.diff(S,1,1)#calculates stock price changes
    #dS1 = np.diff(S,1,1)#calculates stock price changes
    #dS2 = np.diff(S2,1,1)#calculates S2 price changes (Swap)
    #dS = np.concatenate((dS1,dS2),1)
    
    #output used for input
    tSinput = []
    tim = np.linspace(0,T,steps+1)#Vector of timepoint where pf i adjusted.
    
    for i in range(steps):
        timv = np.repeat(tim[i],n)
        timv = np.reshape(timv,(n,1))
        Sv = np.reshape(S[:,i],(n,1))
        #volv = np.reshape(vol[:,i],(n,1))
        tSinput.append(np.concatenate((timv,Sv),1))#Input is (t,S)
    return tSinput,dS

# Black Scholes

In [ ]:
def BlackScholesPath(n):
    xi = npr.normal(0,np.sqrt(dt),(n,steps))#a matrix of N(0,1/steps)
    W = np.apply_along_axis(np.cumsum,1,xi)#makes BM from N(0,1/steps)
    W = np.concatenate((np.zeros((n,1)),W),1)#adds zeros for first entry
    drift = dt*np.linspace(0,mu+sigma**2/2,steps+1)#drift vector
    drift = np.reshape(drift,(1,steps+1))#it changes data format
    drift = np.repeat(drift,n,axis=0)#makes a matrix with same drift in all rows.
    S = S0 * np.exp(drift + sigma * W)#makes stock paths
    return S

#only stock
def DataInputOutput_BS(S):
    dS = np.diff(S,1,1)#calculates stock price changes
    tim = np.linspace(0,T,steps+1)#Vector of timepoint where pf i adjusted.
    tSinput = []
    for i in range(steps):
        timv = np.repeat(tim[i],n)
        timv = np.reshape(timv,(n,1))
        logSv = np.log(np.reshape(S[:,i],(n,1)))
        tSinput.append(np.concatenate((timv,logSv),1))
    return tSinput,dS